<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/DataModelAntoineEquation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from lxml import html
import requests
import jax
import jax.numpy as jnp
import numpy as np
from scipy.optimize import minimize
from plotly.subplots import make_subplots
import plotly.io as pio
from collections import deque
pio.templates.default='plotly_dark'

In [ ]:
original_url='http://www.ddbst.com/en/EED/PCP/VAP_C174.php'

url = 'https://raw.githubusercontent.com/profteachkids/CHE2064/master/DDBWaterVaporPressure.html'

content= requests.get(url).content
tree=html.fromstring(content)

In [ ]:
T_strings=tree.xpath('//*[@id="online-ddb-form"]/div[2]/table[3]/tbody/tr/td[position()<3]/text()')

In [ ]:
data = np.array([(T_string[0], T_string[1]) for T_string in zip(T_strings[::2],T_strings[1::2])]).astype(np.float32)

In [ ]:
T=data[:,0]
log_Pv = jnp.log10(1000*data[:,1])

In [ ]:
fig = make_subplots(rows=1,cols=1)
fig.add_scatter(x=T, y=log_Pv, mode='markers', marker_size=8, marker_color='rgba(0,0,0,0)',
                marker_line_color='rgb(0,0,255)', marker_line_width=1, name='Data')
fig.update_layout(width=800, height=500)
fig.update_layout(xaxis_title='T (K)', yaxis_title='$Log_{10} P\ (Pa)$a')
fig.show()

In [ ]:
def model(a, b, c, T):
  return a-b/(T+c)

In [ ]:
a_guess = 10
b_guess = 2000.0
c_guess = 0.0
x0 = [a_guess, b_guess, c_guess]

def sqerror(x, T, log_Pv):
  model_Pv = model(x[0],x[1],x[2], T)
  err = jnp.sum( (model_Pv-log_Pv)**2)
  return err

def sqerror_scaled(x, T, log_Pv):
  model_Pv = model(x[0],x[1]*1000,x[2]*100, T)
  err = jnp.sum( (model_Pv-log_Pv)**2)
  return err

# res = minimize(sqerror, x0=x0, args=(T, log_Pv), tol=1e-10)
# res = minimize(sqerror, x0=x0, args=(T, log_Pv), tol=1e-10, method='COBYLA')
res = minimize(sqerror, x0=x0, args=(T, log_Pv), tol=1e-10, jac=jax.grad(sqerror))

In [ ]:
res

      fun: 0.007196569349616766
 hess_inv: array([[ 7.58694502e-02, -6.82264613e-01, -5.39620685e+00],
       [-6.82264613e-01,  2.72361035e+01,  8.30982688e+01],
       [-5.39620685e+00,  8.30982688e+01,  5.19189437e+02]])
      jac: DeviceArray([-3.0040741e-04,  5.0894785e-05, -6.6345092e-06], dtype=float32)
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 170
      nit: 9
     njev: 156
   status: 2
  success: False
        x: array([  10.5078807 , 2001.11890403,  -11.32763292])

In [ ]:
T_model = jnp.linspace(T[0],T[-1],100)
fig.add_scatter(x=T_model, y=model(*res.x, T_model), mode='lines', line_width=1, name='Antoine')
fig.show()